## Imports

In [1]:
import keyboard
import mss
import cv2
import numpy
from time import time, sleep
import pyautogui
import numpy as np
import glob
import time
import os
import matplotlib.pyplot as plt

## Helper Functions

In [17]:
def load_video(filepath=""):
    cap = cv2.VideoCapture(filepath)

    while(cap.isOpened()):
        ret, frame = cap.read()
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [2]:
def match_template(image, temp_path, meth, bound=[0,1080,0,1920]):
    """
    params:
    image: base image to match
    temp_path: path to template image or image itself
    meth: matching method from ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
                'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
    bound: bounds of original image to splice and match on
    return: list of locations for matches
    """
    if type(temp_path) == str:
        img = cv2.imread(image)
    else:
        img = image
        
    # splice and copy image based on bounds
    img2 = img.copy()[bound[0]:bound[1],bound[2]:bound[3]] # [120:200, 900:1050]

    # read template and get shape
    template = cv2.imread(temp_path)
    h, w = template.shape[:-1]
    
    # evalute method to feed matchTemplate
    method = eval(meth)
    
    # Apply template Matching
    res = cv2.matchTemplate(img2,template,method)
    
    # apply threshold to matches
    threshold = 0.70
    loc = np.where( res >= threshold)

    return loc

In [8]:


def collect_boards_every_nth(video_path, n, verbose=False):
    
    time_last = time.time()
    
    # boolean flags for combat and planning
    combat_flag = False
    planning_flag = False
    
    # intialize counters
    frame_count = 0 # counts total frames
    round_counter = 0 # count for rounds
    creep_lst = [1,2,3,9,15,21,27,33]
    
    #timing counterrs
    counter_threshold = 170
    planning_counter_threshold = 480
    
    # start video capture
    cap = cv2.VideoCapture(video_path)
    
    while(cap.isOpened()):
        # grab latest frame without rendering
        ret = cap.grab()
        
        if frame_count % n == 0 and combat_flag == False:
            
            # render frame
            ret, frame = cap.retrieve()
            
            if frame is None:
                break
            
            # default bound for combat reference
            bound = [120,200, 900,1050] 
            loc = match_template(frame, r"images/reference/combat.png",'cv2.TM_CCOEFF_NORMED', bound) # match combat ref
            
            # if match list is not empty
            if len(loc[0]) != 0:
                for pt in zip(*loc[::-1]):
                    round_counter += 1
                    combat_counter = 0
                    combat_flag = True
                    
                    if round_counter in creep_lst:
                        counter_threshold = 170
                    else:
                        counter_threshold = 310    
                    break
                    
            # default bound for planning reference
            bound = [120,200, 800,1200]
            loc = match_template(frame, r"images/reference/planning.png",'cv2.TM_CCOEFF_NORMED', bound)
            
            if len(loc[0]) != 0 and planning_flag == False:

                planning_flag = True
                planning_counter = 0
                cv2.imwrite(r"images/boards/planning/unprocessed/combat_board_" + str(match_counter)+ ".png", frame)

        # trigger save image for combat board            
        if combat_flag and combat_counter == counter_threshold:
            ret, frame = cap.retrieve()
            combat_flag = False
            cv2.imwrite(r"images/boards/combat/combat_board_" + str(match_counter)+ ".png", frame)
            
            if verbose:
                time_now = time.time()
                print("saving combat: ", match_counter)
                print("Time between saves: {:.2f}ms".format(time_now - time_last))
                
        # trigger save image for planning board  
        if planning_flag and planning_counter >= planning_counter_threshold:
            ret, frame = cap.retrieve()
            planning_flag = False
            cv2.imwrite(r"images/boards/planning/planning_board_" + str(match_counter)+ ".png", frame)
            
            if verbose:
                time_now = time.time()
                print("saving combat: ", match_counter)
                print("Time between saves: {:.2f}ms".format(time_now - time_last))

        # quit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        
        frame_count+=1
        
        if combat_flag:
            combat_counter += 1
        if planning_flag:
            planning_counter += 1
            
    cap.release()
    cv2.destroyAllWindows()

In [18]:
load_video(r"videos/unprocessed/video2.mp4")

In [9]:
start_time = time.time()

collect_boards_every_nth(r"videos/unprocessed/video2.mp4", 15, True)
print("[INFO] elasped time: {:.2f}ms".format(time.time() - start_time))

saving combat:  1
Time between saves: 10.27ms
saving combat:  1
Time between saves: 15.59ms
saving combat:  2
Time between saves: 16.83ms
saving combat:  2
Time between saves: 22.12ms
saving combat:  3
Time between saves: 24.56ms
saving combat:  3
Time between saves: 29.06ms
saving combat:  4
Time between saves: 37.75ms
saving combat:  4
Time between saves: 48.69ms
saving combat:  5
Time between saves: 54.15ms
saving combat:  5
Time between saves: 62.48ms
saving combat:  6
Time between saves: 67.59ms
saving combat:  6
Time between saves: 84.53ms
saving combat:  7
Time between saves: 88.54ms
saving combat:  7
Time between saves: 96.61ms
saving combat:  8
Time between saves: 101.56ms
saving combat:  9
Time between saves: 110.13ms
saving combat:  9
Time between saves: 117.01ms
saving combat:  10
Time between saves: 122.57ms
saving combat:  10
Time between saves: 132.77ms
saving combat:  11
Time between saves: 141.20ms
saving combat:  11
Time between saves: 150.92ms
saving combat:  12
Time

## Deprecated